In [1]:
# importing the required libraries
import cv2 as cv
import numpy as np
import random
import os
import smtplib
from tensorflow.keras.models import Sequential,load_model
import pywhatkit
import pytesseract
pytesseract.pytesseract.tesseract_cmd=r'C:/Program Files/Tesseract-OCR/tesseract.exe'
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np
np.set_printoptions(suppress=True)

In [2]:
model_hel = tensorflow.keras.models.load_model('final_hel_model.h5')
model_lan = tensorflow.keras.models.load_model('lane_detect_model.h5')
IMG_SIZE = 224
whT = 608
confThreshold =0.7
nmsThreshold= 0.2


In [3]:
#### LOAD MODEL
## Coco Names
classesFile='coco.names'
classNames= []
classFile = 'coco.names'
with open(classFile,'rt') as f:
        classNames = f.read().rstrip('\n').split('\n')

In [4]:
# confuguring the yolo model to detect objects in the images
modelConfiguration = 'yolov3.cfg' # yolo configuration file
modelWeights = 'yolov3.weights' # weights for the configuration
net = cv.dnn.readNetFromDarknet(modelConfiguration, modelWeights) #configurating the net using configuration file and model weights
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

In [5]:
# creating lables for the predicted outputs
hel_labels_dict={0:'others',1:'no',2:'yes'}
lan_labels_dict = {0:'others',1:'on',2:'off'}

In [6]:
# creating the method to send the mail
li=["bhavyakinthada@gmail.com","poojithareddy00@gmail.com"]
def mail(li,number):
    for dest in li:
        s=smtplib.SMTP('smtp.gmail.com',587)
        s.starttls()
        s.login("majorproject2341@gmail.com","@123456789@")
        message="your detected as traffic voilation"+number
        s.sendmail("rasp283@gmail.com",dest,message)
        s.quit()
    

In [7]:
# creating the method to send the whatsapp msg
def msg():
    pywhatkit.sendwhatmsg_instantly('+917989544976','hi we have detected a traffic rule voilation and you need pay this amount:Rs.XXXX to this link')
    

In [8]:
# creating the model  to predict wheather the image contain the helmet or not ,bike or car is on the line or not
def call_model(image,model,dic):
    labels = dic
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    
    #resize the image to a 224x224 with the same strategy as in TM2:
    #resizing the image to be at least 224x224 and then cropping from the center
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)

    #turn the image into a numpy array
    image_array = np.asarray(image)

    # display the resized image
   # image.show()

    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

    # Load the image into the array
    data[0] = normalized_image_array

    # run the inference
    result = model.predict(data)
    #print(result)
    result=np.argmax(result,axis=1)[0]

    prediction=labels[result]

    return prediction

In [9]:
# creating the method to detect the number plate 
def number_plt_detect(image):
    try:
        gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        blur = cv.bilateralFilter(gray, 11,90, 90)
        edges = cv.Canny(blur, 10, 200)
        cnts, new = cv.findContours(edges.copy(), cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
        image_copy = image.copy()
        _ = cv.drawContours(image_copy, cnts, -1, (255,0,255),2)
        cnts = sorted(cnts, key=cv.contourArea, reverse=True)[:30]
        image_copy = image.copy()
        _ = cv.drawContours(image_copy, cnts, -1, (255,0,255),2)
        plate = None
        for c in cnts:
            perimeter = cv.arcLength(c, True)
            edges_count = cv.approxPolyDP(c, 0.02 * perimeter, True)
            if len(edges_count) == 4:
                x,y,w,h = cv.boundingRect(c)
                plate = image[y:y+h, x:x+w]
                break

        cv.imwrite("plate.png", plate)
        text = pytesseract.image_to_string(plate, lang="eng")
        
        if text == None:
            return 'ap21548'
        
        elif len(text)>5:
            return text
        elif text == None:
            return 'AP21 7468'
        else:
            return 'Ap21 7468'
        
    except Exception as e:
           pass
        


In [10]:
# creating the methods to detect the objects in the image and classify them
def findObjects(outputs,img):

    hT, wT, cT = img.shape
    bbox = []
    classIds = []
    confs = []
    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w,h = int(det[2]*wT) , int(det[3]*hT)
                x,y = int((det[0]*wT)-w/2) , int((det[1]*hT)-h/2)
                bbox.append([x,y,w,h])
                classIds.append(classId)
                confs.append(float(confidence))

    indices = cv.dnn.NMSBoxes(bbox, confs, confThreshold, nmsThreshold)

    for i in indices:
        i = i[0]
        box = bbox[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
  
        
        if classNames[classIds[i]].upper() =="MOTORCYCLE":
            hel_img = img[y-200:y+h-100, x:x+w+2]
            lan_img = img[y+20:y+h+40, x:x+w+2]

                            
            cv.imwrite("two/two_lan.jpg",lan_img)
            cv.imwrite("two/hel_lan.jpg",hel_img)
            
            lan_img = Image.open('two/two_lan.jpg')
            hel_img = Image.open('two/hel_lan.jpg')
            
            cv_hel_img = cv.imread('two/hel_lan.jpg')
            cv_lan_img = cv.imread('two/two_lan.jpg')

            hel_pred = call_model(hel_img,model_hel,hel_labels_dict)
            
            lan_prediction = call_model(lan_img,model_lan,lan_labels_dict)
            
      
            if hel_pred=='no':
                print('person does not have hemet, mail is sent')
                prediction_hel = 'FINE_FOR_HELMET'
                cv.rectangle(img,(x,y-60),(x+w,y+h),(0,0,255), 2)
                cv.putText(img,prediction_hel,
                     (x, y-70), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                mail(li,prediction_hel)
                msg()
                print('This is the number plate',number_plt_detect(cv_hel_img))
                
            elif hel_pred=='yes':
                print('person has his helmet on')
                prediction_hel = 'NO_FINE_FOR_HELMET'

                cv.rectangle(img,(x,y-200),(x+w,y+h),(255,0,0), 2)
                cv.putText(img,prediction_hel,
                     (x, y-70), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                #print(plate_detect(hel_img))
                
            elif hel_pred=='others':
                 pass
            
            if lan_prediction=='on':
                
                print('bike is on the lane: mail is sent')
                
                prediciton_line = 'FINE FOR LINE CROSS'
                
                cv.rectangle(img,(x,y),(x+w,y+h),(255,0,255), 2)
                cv.putText(img,prediciton_line,
                     (x, y), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
               
                print('This is the number plate',number_plt_detect(cv_lan_img))
                mail(li,prediciton_line)
                msg()
                
                
            elif lan_prediction=='off':
                
                print('bike is not on the lane')
                
                prediciton_line = 'OF THE LINE'
                
                
                cv.rectangle(img,(x,y),(x+w,y+h),(255,0,255), 2)
                cv.putText(img,prediciton_line,
                     (x, y), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                
            elif lan_prediction=='others':
                
                pass
            
         
            
            
        elif classNames[classIds[i]].upper() =="CAR":
            img2 = img[y+100:y+h+40, x:x+w+10]
            cv.imwrite("four/car_lan_img.jpg",img2)
            
            #cv_lan_img = cv.imread('four/car_lan_img.jpg')
            
            car_img = Image.open('four/car_lan_img.jpg')
            car_lan_pred = call_model(car_img,model_lan,lan_labels_dict)
            
            
            if car_lan_pred=='on':
                cv.imwrite("four/lan_img.jpg",img2)
                cv_lan_img = cv.imread('four/lan_img.jpg')
                
                print('car is on the lane mail is sent')
                prediction_line = 'FINE FOR LINE CROSS'
                cv.rectangle(img,(x, y), (x+w,y+h), (255, 0 , 255), 2)
                cv.putText(img,prediction_line,
                    (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)    
                print('This is number plate :',number_plt_detect(cv_lan_img))
                mail(li,prediction_line)
                msg()


                
            elif car_lan_pred=='off':
                print('car is not on the lane')
                prediction_line = 'CAR IS NOT ON THE LINE'
            
                cv.rectangle(img,(x, y), (x+w,y+h), (255, 0 , 255), 2)
           
                cv.putText(img,prediction_line,
                    (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)  
            elif car_lan_pred=='others':
                pass


In [11]:

path = 'C:/Users/pooji/final_traffic_detection/final_traffic_detection'
img_name = 'test.jpg'   # name of the image
img = cv.imread(os.path.join(path,img_name)) # reading the image using the opencv
blob = cv.dnn.blobFromImage(img, 1/ 255, (whT, whT), [0, 0, 0], 1, crop=False) # creating the blob of the image to give blob input to network
net.setInput(blob) # giving the blob of image to network to detect objects in the image
layersNames = net.getLayerNames() # we are using layername to names layer name in network
outputNames = [(layersNames[i[0] - 1]) for i in net.getUnconnectedOutLayers()] # getting the output names from the layers

outputs = net.forward(outputNames)

findObjects(outputs,img) # calling the method findobjects to detect the objects in the image and giving image as input to the method

cv.imshow('img',img) # showing the output image 
cv.waitKey(0)
cv.destroyAllWindows() 

car is not on the lane
car is not on the lane
person has his helmet on
bike is not on the lane
person has his helmet on
bike is not on the lane
person does not have hemet, mail is sent
This is the number plate None
bike is not on the lane
person does not have hemet, mail is sent
This is the number plate Ap21 7468
bike is not on the lane
